In [ ]:
import pygame
import chess
import os

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 800
SQUARE_SIZE = WIDTH // 8
PIECE_SIZE = int(SQUARE_SIZE)

# Colors
WHITE = (240, 217, 181)
BLACK = (181, 136, 99)
HIGHLIGHT = (255, 255, 0, 100)
RED = (255, 0, 0)

# Load piece images
pieces = {}
for color in ['w', 'b']:
    for piece in ['P', 'N', 'B', 'R', 'Q', 'K']:
        key = color + piece
        img = pygame.image.load(f'pieces/{key}.svg')
        # pieces[key] = pygame.transform.smoothscale(img, (PIECE_SIZE, PIECE_SIZE))
        pieces[key] = pygame.transform.smoothscale_by(img,2)

class ChessGame:
    def __init__(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("PyChess")
        self.board = chess.Board()
        self.selected_piece = None
        self.dragging = False
        self.move_history = []
        self.undo_rect = pygame.Rect(650, 700, 120, 50)
        self.reset_rect = pygame.Rect(650, 600, 120, 50)
        
    def draw_board(self):
        for row in range(8):
            for col in range(8):
                color = WHITE if (row + col) % 2 == 0 else BLACK
                pygame.draw.rect(self.screen, color, 
                               (col*SQUARE_SIZE, row*SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

    def draw_pieces(self):
        for square in chess.SQUARES:
            piece = self.board.piece_at(square)
            if piece:
                row, col = 7 - (square // 8), square % 8
                x = col * SQUARE_SIZE + (SQUARE_SIZE - PIECE_SIZE) // 2
                y = row * SQUARE_SIZE + (SQUARE_SIZE - PIECE_SIZE) // 2
                
                if self.dragging and square == self.selected_piece:
                    continue  # Skip drawing dragged piece here
                
                self.screen.blit(pieces[('w' if piece.symbol().isupper() else 'b') + piece.symbol().upper()], (x, y))
                pygame.draw.rect(self.screen, 'red', pygame.Rect(x,y,PIECE_SIZE,PIECE_SIZE),3)

    def draw_dragged_piece(self, pos):
        if self.selected_piece is not None:
            piece = self.board.piece_at(self.selected_piece)
            x = pos[0] - PIECE_SIZE // 2
            y = pos[1] - PIECE_SIZE // 2
            self.screen.blit(pieces[('w' if piece.symbol().isupper() else 'b') + piece.symbol().upper()], (x, y))

    def draw_buttons(self):
        # Undo button
        pygame.draw.rect(self.screen, RED, self.undo_rect)
        font = pygame.font.Font(None, 36)
        text = font.render("Undo", True, (255, 255, 255))
        self.screen.blit(text, (self.undo_rect.x + 20, self.undo_rect.y + 10))
        
        # Reset button
        pygame.draw.rect(self.screen, RED, self.reset_rect)
        text = font.render("Reset", True, (255, 255, 255))
        self.screen.blit(text, (self.reset_rect.x + 20, self.reset_rect.y + 10))

    def square_from_pos(self, pos):
        col = pos[0] // SQUARE_SIZE
        row = 7 - (pos[1] // SQUARE_SIZE)
        return chess.square(col, row)

    def handle_move(self, from_square, to_square):
        move = chess.Move(from_square, to_square)
        if move in self.board.legal_moves:
            self.board.push(move)
            self.move_history.append(self.board.fen())
            return True
        return False

    def undo_move(self):
        if len(self.move_history) > 1:
            self.move_history.pop()
            self.board = chess.Board(self.move_history[-1])

    def reset_game(self):
        self.board.reset()
        self.move_history = [self.board.fen()]
        self.selected_piece = None
        self.dragging = False

    def run(self):
        clock = pygame.time.Clock()
        running = True
        self.move_history.append(self.board.fen())
        
        while running:
            mouse_pos = pygame.mouse.get_pos()
            self.screen.fill((0, 0, 0))
            self.draw_board()
            self.draw_pieces()
            # self.draw_buttons()


            if self.dragging:
                self.draw_dragged_piece(mouse_pos)

            if self.board.is_game_over():
                font = pygame.font.Font(None, 74)
                text = font.render("Game Over!", True, (0, 0, 0))
                self.screen.blit(text, (WIDTH//2 - 150, HEIGHT//2 - 50))
                pygame.display.flip()
                pygame.time.wait(3000)
                self.reset_game()
                
            pygame.display.flip()
            clock.tick(60)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    # if self.undo_rect.collidepoint(event.pos):
                    #     self.undo_move()
                    # elif self.reset_rect.collidepoint(event.pos):
                    #     self.reset_game()
                    # else:
                    square = self.square_from_pos(event.pos)
                    if self.board.color_at(square) == self.board.turn:
                        self.selected_piece = square
                        self.dragging = True
                
                elif event.type == pygame.MOUSEBUTTONUP and self.dragging:
                    self.dragging = False
                    from_square = self.selected_piece
                    to_square = self.square_from_pos(event.pos)
                    
                    if from_square != to_square:
                        self.handle_move(from_square, to_square)
                    
                    self.selected_piece = None
                
                elif event.type == pygame.MOUSEMOTION and self.dragging:
                    pass  # Handled in main loop

        pygame.quit()

if __name__ == "__main__":
    game = ChessGame()
    game.run()